# Creating a Sampled Dataset

**Learning Objectives**
- Sample the natality dataset to create train/eval/test sets
- Preprocess the data in Pandas dataframe

## Introduction

In this notebook we'll read data from BigQuery into our notebook to preprocess the data within a Pandas dataframe. 

In [1]:
PROJECT = "cloud-training-demos"  # Replace with your PROJECT
BUCKET = "cloud-training-bucket"  # Replace with your BUCKET
REGION = "us-central1"            # Choose an available region for Cloud MLE
TFVERSION = "1.13"                # TF version for CMLE to use

In [2]:
import os
os.environ["BUCKET"] = BUCKET
os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION
os.environ["TFVERSION"] = TFVERSION

In [3]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
    gsutil mb -l ${REGION} gs://${BUCKET}
fi

## Create ML datasets by sampling using BigQuery

We'll begin by sampling the BigQuery data to create smaller datasets.

In [4]:
# Create SQL query using natality data after the year 2000
query_string = """
SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
    publicdata.samples.natality
WHERE
    year > 2000
"""

There are only a limited number of years and months in the dataset. Let's see what the hashmonths are.

We'll call BigQuery but group by the hashmonth and see the number of records for each group. This will enable us to get the correct train/eval/test percentages

In [5]:
from google.cloud import bigquery
bq = bigquery.Client(project = PROJECT)

df = bq.query("SELECT hashmonth, COUNT(weight_pounds) AS num_babies FROM (" 
              + query_string + 
              ") GROUP BY hashmonth").to_dataframe()

print("There are {} unique hashmonths.".format(len(df)))
df.head()

There are 96 unique hashmonths.


,hashmonth,num_babies
0,6244544205302024223,346042
1,7108882242435606404,326736
2,3572456083892950068,304112
3,5742197815970064689,359684
4,7604198770453299557,337126


We can make a query to check if our bucketing values result in the correct sizes of each of our dataset splits and then adjust accordingly

In [6]:
sampling_percentages_query = """
WITH
  -- Get label, features, and column that we are going to use to split into buckets on
  CTE_data AS (
  SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
  FROM
    publicdata.samples.natality
  WHERE
    year > 2000),
  -- Get the counts of each of the unique hashs of our splitting column
  CTE_first_bucketing AS (
  SELECT
    hashmonth,
    COUNT(*) AS num_records
  FROM
    CTE_data
  GROUP BY
    hashmonth ),
  -- Get the number of records in each of the hash buckets
  CTE_second_bucketing AS (
  SELECT
    MOD(hashmonth, {0}) AS bucket_index,
    SUM(num_records) AS num_records
  FROM
    CTE_first_bucketing
  GROUP BY
    MOD(hashmonth, {0})),
  -- Calculate the overall percentages
  CTE_percentages AS (
  SELECT
    bucket_index,
    num_records,
    CAST(num_records AS FLOAT64) / (
    SELECT
      SUM(num_records)
    FROM
      CTE_second_bucketing) AS percent_records
  FROM
    CTE_second_bucketing ),
  -- Choose which of the hash buckets will be used for training and pull in their statistics
  CTE_train AS (
  SELECT
    *,
    "train" AS dataset_name
  FROM
    CTE_percentages
  WHERE
    bucket_index >= 0
    AND bucket_index < {1}),
  -- Choose which of the hash buckets will be used for validation and pull in their statistics
  CTE_eval AS (
  SELECT
    *,
    "eval" AS dataset_name
  FROM
    CTE_percentages
  WHERE
    bucket_index >= {1}
    AND bucket_index < {2}),
  -- Choose which of the hash buckets will be used for testing and pull in their statistics
  CTE_test AS (
  SELECT
    *,
    "test" AS dataset_name
  FROM
    CTE_percentages
  WHERE
    bucket_index >= {2}
    AND bucket_index < {0}),
  -- Union the training, validation, and testing dataset statistics
  CTE_union AS (
  SELECT
    0 AS dataset_id,
    *
  FROM
    CTE_train
  UNION ALL
  SELECT
    1 AS dataset_id,
    *
  FROM
    CTE_eval
  UNION ALL
  SELECT
    2 AS dataset_id,
    *
  FROM
    CTE_test ),
  -- Show final splitting and associated statistics
  CTE_split AS (
  SELECT
    dataset_id,
    dataset_name,
    SUM(num_records) AS num_records,
    SUM(percent_records) AS percent_records
  FROM
    CTE_union
  GROUP BY
    dataset_id,
    dataset_name )
SELECT
  *
FROM
  CTE_split
ORDER BY
    dataset_id
"""

modulo_divisor = 100
train_percent = 80.0
eval_percent = 10.0

train_buckets = int(modulo_divisor * train_percent / 100.0)
eval_buckets = int(modulo_divisor * eval_percent / 100.0)

df = bq.query(sampling_percentages_query.format(modulo_divisor, train_buckets, train_buckets + eval_buckets)).to_dataframe()
df.head()

,dataset_id,dataset_name,num_records,percent_records
0,0,train,27708664,0.832794
1,1,eval,2432411,0.073107
2,2,test,3130839,0.094099


Here's a way to get a well-distributed portion of the data in such a way that the train/eval/test sets do not overlap. 

In [7]:
# Added the RAND() so that we can now subsample from each of the hashmonths to get approximately the record counts we want
train_query = "SELECT * FROM (" + query_string + ") WHERE MOD(hashmonth, 100) < 80 AND RAND() < 0.0005"
eval_query = "SELECT * FROM (" + query_string + ") WHERE MOD(hashmonth, 100) >= 80 AND MOD(hashmonth, 100) < 90  AND RAND() < 0.0005"
test_query = "SELECT * FROM (" + query_string + ") WHERE MOD(hashmonth, 100) >= 90 AND RAND() < 0.0005"

train_df = bq.query(train_query).to_dataframe()
eval_df = bq.query(eval_query).to_dataframe()
test_df = bq.query(test_query).to_dataframe()

print("There are {} examples in the train dataset.".format(len(train_df)))
print("There are {} examples in the validation dataset.".format(len(eval_df)))
print("There are {} examples in the test dataset.".format(len(test_df)))

There are 13955 examples in the train dataset.
There are 1229 examples in the validation dataset.
There are 1544 examples in the test dataset.


## Preprocess data using Pandas

We'll perform a few preprocessing steps to the data in our dataset. Let's add extra rows to simulate the lack of ultrasound. That is we'll duplicate some rows and make the `is_male` field be `Unknown`. Also, if there is more than child we'll change the `plurality` to `Multiple(2+)`. While we're at it, We'll also change the plurality column to be a string. We'll perform these operations below. 

Let's start by examining the training dataset as is.

In [8]:
train_df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,8.562754,True,30,1,41.0,774501970389208065
1,7.561856,False,24,1,39.0,774501970389208065
2,8.686213,False,25,1,42.0,774501970389208065
3,8.260721,True,35,1,39.0,774501970389208065
4,8.249698,True,32,1,36.0,774501970389208065


Also, notice that there are some very important numeric fields that are missing in some rows (the count in Pandas doesn't count missing data)

In [9]:
train_df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hashmonth
count,13943.000000,13955.000000,13955.000000,13858.000000,1.395500e+04
mean,7.222551,27.430240,1.034396,38.600808,4.131916e+18
std,1.331898,6.183919,0.193319,2.592723,2.710216e+18
min,0.507063,13.000000,1.000000,18.000000,1.244589e+17
25%,6.563162,23.000000,1.000000,38.000000,1.525201e+18
50%,7.312733,27.000000,1.000000,39.000000,3.572456e+18
75%,8.062305,32.000000,1.000000,40.000000,6.691862e+18
max,12.015193,50.000000,4.000000,47.000000,9.183606e+18


It is always crucial to clean raw data before using in machine learning, so we have a preprocessing step. We'll define a `preprocess` function below. Note that the mother's age is an input to our model so users will have to provide the mother's age; otherwise, our service won't work. The features we use for our model were chosen because they are such good predictors and because they are easy enough to collect.

In [10]:
import pandas as pd

def preprocess(df):
    # clean up data
    # remove what we don"t want to use for training
    df = df[df.weight_pounds > 0]
    df = df[df.mother_age > 0]
    df = df[df.gestation_weeks > 0]
    df = df[df.plurality > 0]

    # modify plurality field to be a string
    twins_etc = dict(zip([1,2,3,4,5],
                   ["Single(1)", "Twins(2)", "Triplets(3)", "Quadruplets(4)", "Quintuplets(5)"]))
    df["plurality"].replace(twins_etc, inplace = True)

    # now create extra rows to simulate lack of ultrasound
    no_ultrasound = df.copy(deep = True)
    no_ultrasound.loc[no_ultrasound["plurality"] != "Single(1)", "plurality"] = "Multiple(2+)"
    no_ultrasound["is_male"] = "Unknown"

    return pd.concat([df, no_ultrasound])

Let's process the train/eval/test set and see a small sample of the training data after our preprocessing:

In [11]:
train_df = preprocess(train_df)
eval_df = preprocess(eval_df)
test_df = preprocess(test_df)

In [12]:
train_df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,8.562754,True,30,Single(1),41.0,774501970389208065
1,7.561856,False,24,Single(1),39.0,774501970389208065
2,8.686213,False,25,Single(1),42.0,774501970389208065
3,8.260721,True,35,Single(1),39.0,774501970389208065
4,8.249698,True,32,Single(1),36.0,774501970389208065


In [13]:
train_df.tail()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
13950,6.874013,Unknown,30,Single(1),40.0,6637442812569910270
13951,6.062712,Unknown,23,Single(1),39.0,6637442812569910270
13952,7.561856,Unknown,28,Single(1),37.0,6637442812569910270
13953,7.125340,Unknown,20,Single(1),39.0,6637442812569910270
13954,6.902673,Unknown,27,Single(1),40.0,6637442812569910270


Let's look again at a summary of the dataset. Note that we only see numeric columns, so `plurality` does not show up.

In [14]:
train_df.describe()

,weight_pounds,mother_age,gestation_weeks,hashmonth
count,27694.000000,27694.000000,27694.000000,2.769400e+04
mean,7.222601,27.441323,38.605907,4.132496e+18
std,1.332423,6.181724,2.573338,2.708673e+18
min,0.507063,13.000000,18.000000,1.244589e+17
25%,6.563162,23.000000,38.000000,1.525201e+18
50%,7.312733,27.000000,39.000000,3.572456e+18
75%,8.062305,32.000000,40.000000,6.691862e+18
max,12.015193,50.000000,47.000000,9.183606e+18


## Write to .csv files 

In the final versions, we want to read from files, not Pandas dataframes. So, we write the Pandas dataframes out as csv files. Using csv files gives us the advantage of shuffling during read. This is important for distributed training because some workers might be slower than others, and shuffling the data helps prevent the same data from being assigned to the slow workers.

In [15]:
train_df.to_csv(path_or_buf = "train.csv", index = False, header = False)
eval_df.to_csv(path_or_buf = "eval.csv", index = False, header = False)
test_df.to_csv(path_or_buf = "test.csv", index = False, header = False)

In [16]:
%%bash
wc -l *.csv

   2442 eval.csv
   3058 test.csv
  27694 train.csv
  33194 total


In [17]:
%%bash
head *.csv

==> eval.csv <==
6.5697754076,False,36,Single(1),39.0,2962886928991417684
6.686620406459999,False,19,Single(1),43.0,2962886928991417684
6.66236955764,False,30,Single(1),39.0,4329667052416032880
5.1257475915,False,22,Single(1),39.0,6910174677251748583
8.062304921339999,True,23,Single(1),41.0,5742197815970064689
7.1429772888,True,26,Single(1),38.0,4329667052416032880
6.56316153974,True,30,Single(1),39.0,74931465496927487
6.0186197526,True,30,Single(1),37.0,2995620979373137889
8.375361333379999,False,32,Single(1),41.0,8391424625589759186
5.12354296888,False,23,Single(1),34.0,6910174677251748583

==> test.csv <==
6.0627122049999995,False,24,Single(1),43.0,5896567601480310696
9.557039057699999,True,25,Single(1),40.0,5896567601480310696
7.62578964258,False,20,Single(1),37.0,8535133781619919492
9.06320359082,True,25,Single(1),40.0,2599641534368510197
7.5618555866,True,28,Single(1),38.0,8668300920110565493
8.811876612139999,True,24,Single(1),43.0,8387817883864991792
8.99926953484,False,24,Sing

In [18]:
%%bash
tail *.csv

==> eval.csv <==
8.000575487979999,Unknown,24,Single(1),39.0,2995620979373137889
7.25100379718,Unknown,18,Single(1),40.0,74931465496927487
8.11521586422,Unknown,29,Single(1),41.0,74931465496927487
8.000575487979999,Unknown,34,Single(1),38.0,8391424625589759186
9.1271376468,Unknown,32,Single(1),40.0,5742197815970064689
7.12313568522,Unknown,31,Single(1),40.0,5742197815970064689
8.75014717878,Unknown,30,Single(1),39.0,2962886928991417684
5.74965579296,Unknown,15,Single(1),36.0,5742197815970064689
7.06140625186,Unknown,36,Single(1),38.0,6910174677251748583
6.8122838958,Unknown,29,Single(1),40.0,2962886928991417684

==> test.csv <==
5.8091806037,Unknown,38,Single(1),37.0,2599641534368510197
6.0009827716399995,Unknown,20,Single(1),34.0,1088037545023002395
6.66457418026,Unknown,29,Single(1),42.0,8535133781619919492
5.56226287026,Unknown,24,Multiple(2+),36.0,1088037545023002395
7.25100379718,Unknown,31,Single(1),38.0,3095933535584005890
8.0578956761,Unknown,27,Single(1),40.0,85351337816199194

Copyright 2017-2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License